<img src="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png" srcset="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_130 130w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_260 260w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_390 390w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_520 520w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_650 650w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_780 780w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_910 910w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1040 1040w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1170 1170w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1290 1290w" sizes="100vw" width="1290">

<p style='margin-top: 1rem; margin-bottom: 1rem;'>Developed by Sebastian Gingter, Developer Consultant @ <a href='https://thinktecture.com' _target='blank'>Thinktecture AG</a> -- More about me on my <a href='https://thinktecture.com/sebastian-gingter' _target='blank'>profile page</a></p>

# Simple Chat with Message History

This notebook demonstrates how to create a simple chat interface with message history using LangChain. The chat will:
1. Maintain a history of all messages
2. Allow adding new messages
3. Use the history as context for responses
4. Store both user and AI messages in the conversation

## Setup

First, let's install the required packages and set up our environment:

In [ ]:
!pip install -q langchain==0.3.7 langchain-openai==0.2.5 langchain-community==0.3.5

import os
from google.colab import userdata

# Set OVH API key
os.environ["OPENAI_API_KEY"] = userdata.get('OVH_AI_ENDPOINTS_ACCESS_TOKEN')

## Available LLMs

OVH provides several LLM options. Choose one by uncommenting the desired model:

In [ ]:
# Model configuration
TEMPERATURE = 0
MAX_TOKENS = 1500

# OVH AI Endpoints Overview: https://endpoints.ai.cloud.ovh.net/

# Uncomment the model you want to use:

# OVH Mistral-7B-0.2
#MODEL_NAME = "Mistral-7B-Instruct-v0.2"
#BASE_URL = "https://mistral-7b-instruct-v02.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Mixtral 8x22B
#MODEL_NAME = "Mixtral-8x22B-Instruct-v0.1"
#BASE_URL = "https://mixtral-8x22b-instruct-v01.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Llama3-70B
#MODEL_NAME = "Meta-Llama-3-70B-Instruct"
#BASE_URL = "https://llama-3-70b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Llama3-8B
#MODEL_NAME = "Meta-Llama-3-8B-Instruct"
#BASE_URL = "https://llama-3-8b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Llama3.1-70B (default)
MODEL_NAME = "Meta-Llama-3_1-70B-Instruct"
BASE_URL = "https://llama-3-1-70b-instruct.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

# OVH Mamba Codestral 7B
#MODEL_NAME = "mamba-codestral-7B-v0.1"
#BASE_URL = "https://mamba-codestral-7b-v0-1.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1/"

# OVH CodeLlama-13B
#MODEL_NAME = "CodeLlama-13b-Instruct-hf"
#BASE_URL = "https://codellama-13b-instruct-hf.endpoints.kepler.ai.cloud.ovh.net/api/openai_compat/v1"

## Initialize Chat Components

Now we'll set up our chat model and message history:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Initialize the chat model with OVH
chat = ChatOpenAI(
    model=MODEL_NAME,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    base_url=BASE_URL
)

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}")
])

# Create the basic chain
chain = prompt | chat

# Initialize message history
message_history = ChatMessageHistory()

# Create a chain with message history
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

## Chat Interface

Let's create a simple function to handle chat interactions:

In [ ]:
def chat_with_history(user_input: str):
    """Send a message to the chat and get a response"""
    response = chain_with_history.invoke(
        {"input": user_input},
        {"configurable": {"session_id": "demo"}}
    )

    print(f"Assistant: {response.content}")
    print("\nCurrent message history:")
    for msg in message_history.messages:
        print(f"{msg.type.capitalize()}: {msg.content}")

## Try it out!

Let's test our chat with some example interactions:

In [ ]:
chat_with_history("Hi! My name is Alice.")

In [ ]:
chat_with_history("What's my name?")

## Your Turn!

Type your message in the cell below and run it to chat with the AI. The assistant will remember the context of your conversation:

In [ ]:
chat_with_history("Your message here")